<a href="https://colab.research.google.com/github/SRIJAA12/terrain_classification/blob/main/terrain_classifier_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tensorflow matplotlib scikit-learn tensorflow-datasets

# --- IMPORTS ---
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from collections import Counter
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow_datasets as tfds

In [ ]:
# Load EuroSAT RGB dataset
dataset, info = tfds.load("eurosat/rgb", with_info=True, as_supervised=True,
                          split=["train[:80%]", "train[80%:]"])

train_ds, val_ds = dataset
class_names = info.features["label"].names
NUM_CLASSES = info.features["label"].num_classes

print("✅ Dataset Loaded!")
print("📊 Number of Classes:", NUM_CLASSES)
print("🏷️ Class Names:", class_names)


In [ ]:
# Constants
IMG_SIZE = 128
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

# Preprocessing: resize + normalize
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

# Data augmentation block
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.1)
])


In [ ]:
# Apply preprocess and augmentations
train_ds = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE)

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)

# Batch and prefetch
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

print("🛠️ Preprocessing & Augmentation done.")


In [ ]:
# Load MobileNetV2 base
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = True  # Fine-tune enabled

# Freeze early layers
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Final model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
# Optional: Class weights to handle imbalance
label_list = [label.numpy() for _, label in train_ds.unbatch()]
class_counts = Counter(label_list)
total = sum(class_counts.values())
class_weights = {i: total / (len(class_counts) * class_counts[i]) for i in class_counts}


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_terrain_model.keras', save_best_only=True)
]

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks,
    class_weight=class_weights  # only if using class weighting
)


print("🎉 Training Complete and model saved to Drive!")


In [ ]:
# Run predictions
val_images, val_labels = next(iter(val_ds.unbatch().batch(16)))
pred_probs = model.predict(val_images)
pred_labels = tf.argmax(pred_probs, axis=1)


In [ ]:
# Plot predictions vs actual
plt.figure(figsize=(12, 8))
for i in range(16):
    plt.subplot(4, 4, i+1)
    img = val_images[i]
    true = class_names[val_labels[i].numpy()]
    pred = class_names[pred_labels[i].numpy()]
    color = "green" if true == pred else "red"
    plt.imshow(img)
    plt.title(f"P: {pred}\nT: {true}", color=color, fontsize=9)
    plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
# Smart messages per class
terrain_messages = {
    "AnnualCrop": "Crop field detected. Maintain medium altitude.",
    "Forest": "Stable zone. Expect calm wind.",
    "HerbaceousVegetation": "Open vegetation. Suitable for scanning.",
    "Highway": "Road detected. Maintain height and avoid vehicles.",
    "Industrial": "Caution: Obstructions likely. Maintain max altitude.",
    "Pasture": "Livestock zone. Maintain safe distance.",
    "PermanentCrop": "Orchard zone. Monitor tree height.",
    "Residential": "Urban zone. Avoid low flyovers.",
    "River": "Water detected. Be cautious of reflections.",
    "SeaLake": "Large water body. Ensure GPS lock."
}


In [ ]:
# Visualize predictions with messages
plt.figure(figsize=(14, 10))
for i in range(16):
    plt.subplot(4, 4, i+1)
    img = val_images[i]
    pred_idx = pred_labels[i].numpy()
    true_idx = val_labels[i].numpy()
    pred_class = class_names[pred_idx]
    true_class = class_names[true_idx]
    message = terrain_messages[pred_class]
    color = "green" if pred_class == true_class else "red"
    plt.imshow(img)
    plt.title(f"{pred_class}\n{message}", fontsize=7, color=color)
    plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
model.save("/content/drive/MyDrive/best_terrain_model.keras")


In [ ]:
import zipfile

# Path to your model
model_path = "/content/drive/MyDrive/best_terrain_model.keras"
zip_path = "/content/drive/MyDrive/best_terrain_model.zip"

# Zip it
with zipfile.ZipFile(zip_path, 'w') as zipf:
    zipf.write(model_path, arcname="best_terrain_model.keras")

print("Model zipped successfully at:", zip_path)

